In [84]:
import pandas as pd
from data_loading import DataLoader
from preprocess import ConvertToMarkdown, SplitDocuments
from openai_utils import OpenAIEmbeddingClient, GenerateEmbeddings
from pinecone_utils import PineconeIndexClient, IngestEmbeddingsToPinecone
import os
from dotenv import load_dotenv

In [97]:
def GabaritoFormatter(gabarito_raw):
    first_page_content = gabarito_raw[0].lower().split("exame de ordem unificado - ")[1]
    first_page_clean = first_page_content.replace('##','').replace('(*) questão anulada', '')
    lines = first_page_clean.split('\n')
    # Removendo lixo no 1o e último itens da lista
    lines = lines[1:-1]
    lines_trimmed = []
    for line in lines:
        line_trimmed = line.strip()
        lines_trimmed.append(line_trimmed)
    
    questions = []
    answers = []
    # Processa a lista em pares: numeração (linha par) e gabarito (linha ímpar)
    for i in range(0, len(lines_trimmed), 2):
        numbers = lines_trimmed[i].split()
        leters = lines_trimmed[i+1].split()
        
        for number, leter in zip(numbers, leters):
            if leter == '*':
                leter = 'ANULADA'
            questions.append(int(number))
            answers.append(leter)

    # Cria o DataFrame
    df = pd.DataFrame({
        'questao': questions,
        'resposta': answers
    })

    # Ordena por número da questão
    df = df.sort_values(by='questao').reset_index(drop=True)
    return df

In [87]:
prova_oab = ConvertToMarkdown(DataLoader('D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\40_prova.pdf'))
gabarito_oab = ConvertToMarkdown(DataLoader('D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\40_gabarito.pdf'))

In [96]:
first_page_content = gabarito_oab[0].lower().split("exame de ordem unificado - ")[1]
first_page_clean = first_page_content.replace('##','').replace('(*) questão anulada', '')
lines = first_page_clean.split('\n')
lines = lines[1:-1]
print(lines)

['1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20', ' b c a c * c d a b c b d b b b d a b d a', '21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40', ' d b a c a c d d b c d b d d a c b c c b', '41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60', ' c a d c d c b a a c b d b a d c a b d a', '61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80', ' d a c c a b a b c a b a c d d d c c b a', '']


In [99]:
GabaritoFormatter(gabarito_oab)

IndexError: list index out of range